In [ ]:
# import mlflow
# mlflow.set_tracking_uri("https://dagshub.com/sanjaynreddy96/mlops-projects.mlflow")

In [12]:
import os
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/sanjaynreddy96/mlops-projects.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="sanjaynreddy96"
os.environ["MLFLOW_TRACKING_PASSWORD"]="ae8befe1b64d8c21fecc0f1f5931562aa4a2abc5"

In [37]:
import os
%pwd

'c:\\Users\\sanja\\Desktop\\Data_science\\4_MLOps\\mlops-projects\\wine_quality_prediction_pipeline'

In [38]:
from dataclasses import dataclass
from pathlib import Path


@dataclass
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str
    mlflow_uri: str


In [39]:
from src.WineQualityPrediction.constants import *
from src.WineQualityPrediction.utils.common import read_yaml, create_directories,save_json

In [40]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.ElasticNet
        schema = self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config=ModelEvaluationConfig(
            root_dir = config.root_dir,
            test_data_path=config.test_data_path,
            model_path = config.model_path,
            all_params=params,
            metric_file_name = config.metric_file_name,
            target_column = schema.name,
            mlflow_uri="https://dagshub.com/sanjaynreddy96/mlops-projects.mlflow"
        )
        return model_evaluation_config


In [41]:
import os
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import joblib

In [ ]:
from mlflow.models.signature import infer_signature
#from mlflow.sklearn import SklearnModel
import mlflow.pyfunc

class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    def eval_metrics(self, actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        return rmse, mae, r2
    
    def log_into_mlflow(self):
        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        test_x = test_data.drop([self.config.target_column], axis=1)
        test_y = test_data[[self.config.target_column]]

        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        # Tracking in MLFlow
        with mlflow.start_run():
            predicted_qualities = model.predict(test_x)
            (rmse, mae, r2) = self.eval_metrics(test_y, predicted_qualities)
            
            # Saving metrics as local
            scores = {"rmse": rmse, "mae": mae, "r2": r2}
            save_json(path=Path(self.config.metric_file_name), data=scores)

            # Logging parameters and metrics
            mlflow.log_params(self.config.all_params)
            mlflow.log_metric("rmse", rmse)
            mlflow.log_metric("r2", r2)
            mlflow.log_metric("mae", mae)
            
            '''
            # Model registry does not work with file store
            if tracking_url_type_store != "file":
                mlflow.sklearn.log_model(model, "model")
            else:
                mlflow.sklearn.log_model(model, "model")
            '''

            # Log model as artifact
            local_model_path = "artifacts/model_evaluation/sklearn_model"
            os.makedirs(local_model_path, exist_ok=True)
            joblib.dump(model, os.path.join(local_model_path, "model.joblib"))
            mlflow.log_artifacts(local_model_path, artifact_path="model")


In [59]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation = ModelEvaluation(config=model_evaluation_config)
    model_evaluation.log_into_mlflow()
except Exception as e:
    raise e

[2025-06-27 11:51:30,798: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-06-27 11:51:30,816: INFO: common: yaml file: params.yaml loaded successfully]
[2025-06-27 11:51:30,828: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-06-27 11:51:30,836: INFO: common: created directory at: artifacts]
[2025-06-27 11:51:30,843: INFO: common: created directory at: artifacts/model_evaluation]
[2025-06-27 11:51:31,344: INFO: common: json file saved at: artifacts\model_evaluation\metrics.json]
🏃 View run carefree-mouse-431 at: https://dagshub.com/sanjaynreddy96/mlops-projects.mlflow/#/experiments/0/runs/96d559fd73ef4d6095d8a97b1a81fe5b
🧪 View experiment at: https://dagshub.com/sanjaynreddy96/mlops-projects.mlflow/#/experiments/0
